In [2]:
import sys, os

root = os.path.dirname(os.path.abspath(os.curdir))
if root not in sys.path:
    sys.path.insert(1, root)
#os.chdir('c:\\Development\\total-fund-exposure-MS')
import json

import importlib
import pandas as pd
from datetime import datetime, timedelta
from copy import copy
import csv

# import data_acquisition
from utils import sql_dictionary
#from core import positions_analytics_machine as PAM
from core import position_analytics_machine as PAM
from constants import position_analytics_constants as PAC

from psp.ada_connect import AdaConnect
from psp.psp_db.psp_connection import Connection

db_settings_path: str = os.path.join(root + "\config\conn_props.json")

initialized_adaconnect: AdaConnect = AdaConnect(file=db_settings_path, username="oromdhani@investpsp.ca")
conn_sql:Connection = initialized_adaconnect.get("sqlserver")
conn_sql.connect()

conn_snow:Connection = initialized_adaconnect.get("snowflake")
conn_snow.connect()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
os.chdir(root)
evaluation_date = datetime.today()

#find last 3 end of month dates
last_month_last_day = evaluation_date.replace(day=1) - timedelta(days=1)
position_dates = []
for i in range(3):
    # Add the end-of-month date to the list
    position_dates.append(last_month_last_day.strftime('%Y-%m-%d'))
    # Calculate the last day of the month before the current month
    last_month_last_day = last_month_last_day.replace(day=1) - timedelta(days=1)

pam_calculator = PAM.positions_analytics_machine(
    evaluation_date = evaluation_date.strftime('%Y-%m-%d') ,
    position_dates = position_dates,
    psp_portfolio_ids = None,
    force_refresh = False,
    country_to_use = None,
    sql_dictionary = sql_dictionary.sql_dictionary,
    initialized_adaconnect = initialized_adaconnect,
    file_path = "output/",
    verbose = True,
    database_warehouse = "PSP",
    database_mra       = "PSP",
    database_tfe       = "PSP"
    )

In [4]:
#verify data before TFE (data from v1 code)
username = os.getlogin() 
df = pd.read_csv(PAC.exposure_v1_root_folder + username + PAC.exposure_v1_folder_path + PAC.exposure_v1_file_name, sep = '|')
for date in sorted(df['ValuationDate'].unique()):
    sum = df[df['ValuationDate'] == date]['CalculatedIssuerExposureinCAD'].sum()
    print(f'{date} : {sum}')

2021-08-31 : 217836064010.2385
2021-09-30 : 226570271510.5216
2021-10-31 : 224457646064.7682
2021-11-30 : 229954172112.62625
2021-12-31 : 238562253605.6748
2022-01-31 : 234004266718.36447
2022-02-28 : 227761972930.53903
2022-03-31 : 212705934213.6449
2022-04-30 : 209097678214.31458
2022-05-31 : 207957236905.40118
2022-06-30 : 203296417370.03674
2022-07-31 : 205527791538.58023
2022-08-31 : 209128313068.27353
2022-09-30 : 211835671912.6346
2022-10-31 : 213205803358.0042


In [4]:
# verify TFE data
#before update
username = os.getlogin() 
folder_path = PAC.exposure_v1_root_folder + username + PAC.exposure_v1_folder_path
for date in next(os.walk(folder_path))[1]:
    file_path = PAC.exposure_v1_root_folder + username + PAC.exposure_v1_folder_path + date + '//' + PAC.exposure_v1_file_name
    df = pd.read_csv(file_path, sep = '|')
    sum = df[df['ValuationDate'] == date]['CalculatedIssuerExposureinCAD'].sum()
    print(f'{date} : {sum}')

2022-11-30 : 215674843094.5965
2022-12-31 : 217760398452.56564
2023-01-31 : 224346758040.6407
2023-02-28 : 223685442491.10034
2023-03-31 : 228554768380.12546
2023-04-30 : 229642036166.71823
2023-05-31 : 226426001729.9762
2023-06-30 : 222648854680.24136
2023-07-31 : 226673096042.7387
2023-08-31 : 230204841392.35187


In [5]:
pam_calculator.generate_csv_file_exposure_v1()

Acquiring private_markets data...
Acquiring positions data...
Acquiring instruments data...


Exception ignored in: <function SFDictFileCache.__del__ at 0x0000020D521D6DD0>
Traceback (most recent call last):
  File "c:\Users\OROMDHANI\source\total-fund-exposure-MS\.venv\lib\site-packages\snowflake\connector\cache.py", line 577, in __del__
    self._save()
  File "c:\Users\OROMDHANI\source\total-fund-exposure-MS\.venv\lib\site-packages\snowflake\connector\cache.py", line 510, in _save
    with open(tmp_file, "wb") as w_file:
KeyboardInterrupt: 


In [22]:
# verify TFE data
# after update
username = os.getlogin() 
folder_path = PAC.exposure_v1_root_folder + username + PAC.exposure_v1_folder_path
for date in next(os.walk(folder_path))[1]:
    file_path = PAC.exposure_v1_root_folder + username + PAC.exposure_v1_folder_path + date + '//' + PAC.exposure_v1_file_name
    df = pd.read_csv(file_path, sep = '|')
    sum = df[df['ValuationDate'] == date]['CalculatedIssuerExposureinCAD'].sum()
    print(f'{date} : {sum}')

2022-11-30 : 215674843094.5965
2022-12-31 : 217760398452.56564
2023-01-31 : 224346758040.6407
2023-02-28 : 223536610038.36755
2023-03-31 : 228495238991.03314
2023-04-30 : 230024533312.8916
